In [ ]:
# Task 2: Time Series Forecasting for Tesla's Stock Price

# Import Libraries
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from pmdarima import auto_arima
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Load Data
data = yf.download('TSLA', start='2015-01-01', end='2024-10-31')
data = data['Close']  # Focusing on closing prices for forecasting
data = data.dropna()

# Split Data into Training and Testing Sets (80-20 split)
train_size = int(len(data) * 0.8)
train, test = data[:train_size], data[train_size:]

# Function to Calculate Evaluation Metrics
def evaluate_forecast(test, predictions):
    mae = mean_absolute_error(test, predictions)
    rmse = sqrt(mean_squared_error(test, predictions))
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    print(f'MAE: {mae}, RMSE: {rmse}, MAPE: {mape}')
    return mae, rmse, mape

# 1. ARIMA Model
# Use auto_arima to find best (p,d,q) parameters
arima_model = auto_arima(train, seasonal=False, trace=True, error_action='ignore', suppress_warnings=True)
arima_order = arima_model.order

# Train ARIMA Model
arima = ARIMA(train, order=arima_order)
arima_fit = arima.fit()

# Make Predictions
arima_predictions = arima_fit.forecast(steps=len(test))
evaluate_forecast(test, arima_predictions)

# Plot Predictions
plt.figure(figsize=(12, 6))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(test.index, arima_predictions, label='ARIMA Predictions')
plt.title("ARIMA Model Forecast")
plt.legend()
plt.show()

# 2. SARIMA Model
# Using auto_arima to find (p,d,q)(P,D,Q,s) for seasonal data
sarima_model = auto_arima(train, seasonal=True, m=12, trace=True, error_action='ignore', suppress_warnings=True)
sarima_order, sarima_seasonal_order = sarima_model.order, sarima_model.seasonal_order

# Train SARIMA Model
sarima = SARIMAX(train, order=sarima_order, seasonal_order=sarima_seasonal_order)
sarima_fit = sarima.fit(disp=False)

# Make Predictions
sarima_predictions = sarima_fit.forecast(steps=len(test))
evaluate_forecast(test, sarima_predictions)

# Plot Predictions
plt.figure(figsize=(12, 6))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(test.index, sarima_predictions, label='SARIMA Predictions')
plt.title("SARIMA Model Forecast")
plt.legend()
plt.show()

# 3. LSTM Model
# Prepare data for LSTM (reshape into 3D array)
train_data_gen = TimeseriesGenerator(train.values, train.values, length=60, batch_size=32)
test_data_gen = TimeseriesGenerator(test.values, test.values, length=60, batch_size=32)

# Define LSTM Model
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(60, 1)))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(units=50, return_sequences=False))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1))

# Compile and Train LSTM Model
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(train_data_gen, epochs=20, validation_data=test_data_gen, verbose=1)

# Make Predictions
lstm_predictions = lstm_model.predict(test_data_gen)
evaluate_forecast(test[60:], lstm_predictions)

# Plot LSTM Predictions
plt.figure(figsize=(12, 6))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(test.index[60:], lstm_predictions, label='LSTM Predictions')
plt.title("LSTM Model Forecast")
plt.legend()
plt.show()
